# 快速开始

## 哲学

Transformers是为寻求使用/研究/扩展大型变压器模型的NLP研究人员而建的自有图书馆。

该库在设计时考虑了两个强烈的目标：

- 尽可能容易且快速地使用：
  - 我们严格限制了要学习的面向用户抽象的数量，实际上几乎没有抽象，使用每个模型只需要三个标准类：配置，模型和令牌生成器，
  - 所有这些类都可以使用通用的`from_pretrained()`实例化方法，以简单统一的方式从预训练的实例中初始化，该方法将负责从库或您自己提供的预训练实例中下载（如果需要），缓存和加载相关类。保存的实例。
  - 因此，该库不是神经网络构建模块的模块化工具箱。如果要扩展/构建该库，只需使用常规的Python / PyTorch模块并从该库的基类继承即可重用模型加载/保存之类的功能。
- 提供性能与原始模型尽可能接近的最新模型：
  - 对于每种架构，我们至少提供一个示例，该示例再现了该架构的正式作者提供的结果，
  - 该代码通常尽可能地接近原始代码库，这意味着某些PyTorch代码可能不像转化为TensorFlow代码那样具有*自爆性*。

其他一些目标：

- 尽可能一致地公开模型的内部：
  - 我们使用单个API授予访问权限的全部隐藏状态和关注权重，
  - 标记器和基本模型的API已标准化，可以在模型之间轻松切换。
- 纳入主观选择的有前途的工具，以对这些模型进行微调/研究：
  - 一种简单/一致的方法，可以向词汇表和嵌入物中添加新标记以进行微调，
  - 遮盖和修剪变压器头的简单方法。

## 主要概念

该库针对每种模型围绕三种类型的类构建：

- **模型类**是`torch.nn.Modules`库中当前提供的8种模型架构的PyTorch模型（），例如`BertModel`
- **配置类**，用于存储构建模型所需的所有参数，例如`BertConfig`。您不必总是实例化这些自身，尤其是如果您使用的是经过预训练的模型而未进行任何修改，则创建模型将自动照顾实例化配置（这是模型的一部分）
- **标记程序类**，用于存储每个模型的词汇表，并提供用于编码/解码要嵌入模型的标记嵌入索引列表中的字符串的方法，例如`BertTokenizer`

所有这些类都可以从经过预训练的实例中实例化，并使用两种方法在本地保存：

- `from_pretrained()`让你实例/配置/标记者从库本身可能提供了预训练版本（目前27款是作为上市模式[在这里](https://huggingface.co/transformers/pretrained_models.html)）由用户或存储在本地（或服务器上），
- `save_pretrained()`使您可以在本地保存模型/配置/令牌，以便可以使用来重新加载它`from_pretrained()`。

我们将通过一些简单的快速入门示例来结束本快速入门之旅，以了解如何实例化和使用这些类。本文档的其余部分分为两部分：

- “ **主要类别”**部分详细介绍了三种主要类别（配置，模型，令牌生成器）的常见功能/方法/属性，以及一些作为培训实用程序提供的与优化相关的类别，
- 该**装基准**部分的所有细节每个班级每个模型架构的变体，特别是呼吁他们每个人在输入/输出，你应该期望。

## 快速导览：用法

这是两个示例，展示了一些`Bert`和`GPT2`类以及预训练的模型。

有关每个模型类的示例，请参见完整的API参考。

### BERT示例

让我们从使用文本字符串准备标记化的输入（将要馈送到Bert的标记嵌入索引的列表）开始 `BertTokenizer`

In [1]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

ImportError: dlopen(/Users/modongsong/.pyenv/versions/miniconda3-4.3.30/envs/transformers/lib/python3.7/site-packages/torch/_C.cpython-37m-darwin.so, 9): Symbol not found: _mkl_blas_caxpy
  Referenced from: /Users/modongsong/.pyenv/versions/miniconda3-4.3.30/envs/transformers/lib/python3.7/site-packages/torch/lib/../../../../libmkl_intel_lp64.dylib
  Expected in: flat namespace
 in /Users/modongsong/.pyenv/versions/miniconda3-4.3.30/envs/transformers/lib/python3.7/site-packages/torch/lib/../../../../libmkl_intel_lp64.dylib

让我们看看如何使用`BertModel`隐藏状态对输入进行编码：

In [2]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    # Transformers models always output tuples.
    # See the models docstrings for the detail of all the outputs
    # In our case, the first element is the hidden state of the last layer of the Bert model
    encoded_layers = outputs[0]
# We have encoded our input sequence in a FloatTensor of shape (batch size, sequence length, model hidden dimension)
assert tuple(encoded_layers.shape) == (1, len(indexed_tokens), model.config.hidden_size)

NameError: name 'BertModel' is not defined

In [3]:
以及如何用于`BertForMaskedLM`预测屏蔽令牌：

SyntaxError: invalid syntax (<ipython-input-3-57b472d13483>, line 1)

In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'